### 将以下代码直接copy进相应Google Image页面的 Inspect-->Console中，urls.txt中保存了该页面中的全部图片链接
  
```
// pull down jquery into the JavaScript console
var script = document.createElement('script');
script.src = "https://ajax.googleapis.com/ajax/libs/jquery/2.2.0/jquery.min.js";
document.getElementsByTagName('head')[0].appendChild(script);
// grab the URLs
var urls = $('.rg_di .rg_meta').map(function() { return JSON.parse($(this).text()).ou; });
```
```
// write the URls to file (one per line)
var textToSave = urls.toArray().join('\n');
var hiddenElement = document.createElement('a');
hiddenElement.href = 'data:attachment/text,' + encodeURI(textToSave);
hiddenElement.target = '_blank';
hiddenElement.download = 'urls.txt';
hiddenElement.click();
```

### 用下面的爬虫代码爬取urls.txt中全部的图片链接，将图片下载到本地

In [ ]:
# -*- coding: utf-8 -*-
# download_images.py

from imutils import paths
import argparse
import requests
import cv2
import os
import time

rows = open('urls.txt').read().strip().split("\n")
total = 0

# loop the URLs
for url in rows:
    try:
        # try to download the image
        print(str(url))
        r = requests.get(url)
        time.sleep(5)
        # save the image to disk
        p = os.path.sep.join([args["output"], "{}.jpg".format(str(total).zfill(8))])
        with open(p, 'wb') as f:
            f.write(r.content)

        # update the counter
        print("[INFO] downloaded: %d"%total)
        total += 1
        
    # handle if any exceptions are thrown during the download process
    except:
        print("[INFO] error downloading {}...skipping".format(p))

# loop over the image paths we just downloaded
for imagePath in paths.list_images(args["output"]):
    # initialize if the image should be deleted or not
    delete = False

    # try to load the image
    try:
        image = cv2.imread(imagePath)

        # if the image is `None` then we could not properly load it
        # from disk, so delete it
        if image is None:
            delete = True

    # if OpenCV cannot load the image then the image is likely
    # corrupt so we should delete it
    except:
        print("Except")
        delete = True

    # check to see if the image should be deleted
    if delete:
        print("[INFO] deleting {}".format(imagePath))
        os.remove(imagePath)

### Run the script, make sure that the "output file path" is already existed
$ python download_images.py --urls urls.txt --output images/santa